In [1]:
%%capture
!pip install textacy
!pip install faiss-gpu
!pip install chromadb
!pip install nltk

In [2]:
%%capture
# Initiliase Ollama server on a thread

#Download ollama
!curl -fsSL https://ollama.com/install.sh | sh
import subprocess
process = subprocess.Popen("ollama serve", shell=True) #runs on a different thread
#Download model
!ollama pull llama3
!pip install ollama
import ollama
model_name = 'llama3'

In [3]:
import re
import asyncio
import pandas as pd
import numpy as np
from pydantic import BaseModel
from ollama import AsyncClient
import textacy.datasets
import requests
import json
from sentence_transformers import SentenceTransformer
from textwrap import shorten
import nltk
from textacy.representations.vectorizers import Vectorizer
from sklearn.metrics.pairwise import cosine_similarity
import chromadb
import torch

In [4]:
# Load Supreme Court dataset
sc = textacy.datasets.SupremeCourt()
sc.download()

# Area code classification for 15 labels
# print(sc.issue_area_codes)

# Create a DataFrame from the dataset
cases = [{"text": record.text} for record in sc.records()]
cases_df = pd.DataFrame(cases)

# Display the first few rows of the dataset
cases_df.head()
# cases_df = cases_df[:100]

100%|██████████| 111M/111M [00:01<00:00, 86.7MB/s] 


,text
0,[ Halliburton Oil Well Cementing Co. v. Walker...
1,"Rehearing Denied Dec. 16, 1946. See . Mr.Claud..."
2,"Rehearing Denied Dec. 16, 1946\n See .\n Appea..."
3,"Mr.\nWalter J. Cummings, Jr., of Washington, D..."
4,"Mr.A. Devitt Vaneck, of Washington, D.C., for ..."


In [5]:
# Get words from model's perpective
response = ollama.chat(model=model_name, messages=[
  {
    'role': 'user',
    'content': '''Give me a list of words and phrases (atleast 15) which would be present in a case text to classify it into following categories. Do not write any explaination or warnings.
            - Criminal Procedure: Cases involving the process of investigating and prosecuting crimes.
            - Civil Rights: Cases about the rights of individuals to receive equal treatment.
            - First Amendment: Cases addressing issues related to freedom of speech, religion, or press.
            - Due Process: Cases focusing on legal safeguards ensuring fair treatment.
            - Privacy: Cases dealing with the right to privacy.
            - Attorneys: Cases about the legal profession and lawyer regulations.
            - Unions: Cases concerning labor unions and collective bargaining.
            - Economic Activity: Cases involving business, trade, or commerce.
            - Judicial Power: Cases addressing the powers of courts and the judiciary.
            - Federalism: Cases about the division of power between state and federal governments.
            - Interstate Relations: Cases about interactions between states.
            - Federal Taxation: Cases involving federal tax laws.
            - Miscellaneous: Cases that do not clearly fit into any of the above categories.
            - Private Action: Cases involving disputes between private individuals or entities.

    Format the output in the following way:
    'Category': words, phrase ...
      ''',
  },
])

In [6]:
# print(response['message']['content'])

def parse_text_to_dict(text):
    result_dict = {}
    lines = text.strip().split('\n')
    
    for line in lines:
        line = line.strip()
        if not line or ':' not in line:
            continue  # Ignore headers or tails
        match = re.match(r"^(.*?):\s*(.*)$", line)
        if match:
            category = match.group(1).strip()
            values = [word.strip() for word in match.group(2).split(',')]
            result_dict[category] = values
    
    return result_dict

def clean_text(text):
    if not isinstance(text, str):
        text = str(text)  # Ensure input is a string
    return re.sub(r'[^a-zA-Z0-9 ]', '', text).strip().lower()

def filter_result_dict(result_dict, valid_categories):
    cleaned_valid_categories = {clean_text(str(value)) for value in valid_categories.values()}
    cleaned_result_dict = {clean_text(key): values for key, values in result_dict.items()}
    return {key: values for key, values in cleaned_result_dict.items() if key in cleaned_valid_categories}

# Convert text to dictionary
result = parse_text_to_dict(response['message']['content'])
# for i in result:
#     print(i)
#     print(result[i])

filtered_result = filter_result_dict(result, sc.issue_area_codes)
for i in filtered_result:
    print(i)
    print(filtered_result[i])
    

criminal procedure
['arrest warrant', 'prosecution', 'indictment', 'conviction', 'trial', 'evidence', 'defendant', 'perpetrator']
civil rights
['equality', 'discrimination', 'racial bias', 'segregation', 'affirmative action', 'equal protection']
first amendment
['freedom speech', 'religious liberty', 'press freedom', 'censorship', 'prior restraint', 'libel']
due process
['procedural fairness', 'substantive justice', 'notice', 'hearing', 'appeal', 'ex post facto law']
privacy
['surveillance', 'privacy violation', 'data breach', 'confidential information', 'secrecy']
attorneys
['lawyer-client privilege', 'attorney-client confidentiality', 'legal ethics', 'malpractice', 'disbarment']
unions
['collective bargaining', 'labor disputes', 'strike', 'lockout', 'union membership', 'fair labor practices']
economic activity
['antitrust', 'intellectual property', 'commercial speech', 'consumer protection', 'market regulation']
judicial power
['judicial review', 'judicial activism', 'jurisdictional 

In [7]:
# Convert dataset records to a DataFrame
records = list(sc.records())
df = pd.DataFrame(records)

# Display the first few rows
print(df.head())

                                                text  \
0  [ Halliburton Oil Well Cementing Co. v. Walker...   
1  Rehearing Denied Dec. 16, 1946. See . Mr.Claud...   
2  Rehearing Denied Dec. 16, 1946\n See .\n Appea...   
3  Mr.\nWalter J. Cummings, Jr., of Washington, D...   
4  Mr.A. Devitt Vaneck, of Washington, D.C., for ...   

                                                meta  
0  {'issue': '80180', 'issue_area': 8, 'n_min_vot...  
1  {'issue': '10500', 'issue_area': 1, 'n_min_vot...  
2  {'issue': '80250', 'issue_area': 8, 'n_min_vot...  
3  {'issue': '20150', 'issue_area': 2, 'n_min_vot...  
4  {'issue': '80060', 'issue_area': 8, 'n_min_vot...  


In [10]:
# Download necessary resources
nltk.download("stopwords")

# def classify_case_rag(case_text, classification_dict):
#     """
#     Dynamically classify a case using Retrieval-Augmented Generation (RAG) by querying relevant legal topics
#     from all categories in the dictionary.
    
#     Args:
#         case_text (str): The text of the case to be classified.
#         classification_dict (dict): Dictionary mapping classification categories to relevant keywords.
    
#     Returns:
#         str: The best-matching class name for the case.
#     """
    
#     # Load the embedding model
#     model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
    
#     # Define the ChromaDB path
#     chroma_db_path = "./chroma_db"
    
#     # Initialize a persistent ChromaDB client
#     chroma_client = chromadb.PersistentClient(path=chroma_db_path)
    
#     # Check if the collection already exists
#     existing_collections = chroma_client.list_collections()
#     if "temp_case_classification" in [col for col in existing_collections]:
#         collection = chroma_client.get_collection("temp_case_classification")
#         print(0)
#     else:
#         collection = chroma_client.create_collection(name="temp_case_classification")

#         # Populate ChromaDB dynamically with classification categories and keywords
#         for class_name, keywords in classification_dict.items():
#             for keyword in keywords:
#                 embedding = model.encode(keyword).tolist()
#                 collection.add(embeddings=[embedding], metadatas=[{"class_name": class_name}], ids=[keyword])
    
#     # Encode the case text for retrieval
#     case_embedding = model.encode(case_text).tolist()
    
#     # Retrieve the most relevant legal categories
#     results = collection.query(query_embeddings=[case_embedding], n_results=3)
    
#     retrieved_classes = [
#         res["class_name"] for res in results["metadatas"][0]
#     ] if results["metadatas"] else []
    
#     if not retrieved_classes:
#         return "No relevant classification found"
    
#     # Prepare relevant context for LLaMA 3
#     retrieved_text = "\n".join(retrieved_classes)

#     prompt = f"""
#     You are an AI assistant trained in Supreme Court case classification. Given the following retrieved categories,
#     classify the new case into the most appropriate issue area.

#     Retrieved Relevant Categories:
#     {retrieved_text}



#     Please reply with the category only and nothing else."""

#     #  Case Text:
#     # {case_text[:1000]}  # Limiting case text for concise processing
    
#     # Query LLaMA 3 using the Ollama instance
#     response = ollama.chat(model='llama3', messages=[{"role": "user", "content": prompt}])
#     # Explicitly delete case_embedding to free memory
#     del case_embedding
#     torch.cuda.empty_cache()  # Clear unused memory from GPU
    
#     return response['message']['content']


import torch
from sentence_transformers import SentenceTransformer

def classify_case_rag(case_text, classification_dict):
    """
    Dynamically classify a case using Retrieval-Augmented Generation (RAG) by querying relevant legal topics.
    
    Args:
        case_text (str): The text of the case to be classified.
        classification_dict (dict): Dictionary mapping classification categories to relevant keywords.
    
    Returns:
        str: The best-matching class name for the case.
    """
    
    # Check GPU availability
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Load the embedding model and move it to GPU
    model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2").to(device)

    # If multiple GPUs, use DataParallel
    if torch.cuda.device_count() > 1:
        print(f"Using {torch.cuda.device_count()} GPUs for inference.")
        model = torch.nn.DataParallel(model)
    
    # Function to handle encoding with multi-GPU support
    def encode_text(text):
        if torch.cuda.device_count() > 1:
            return model.module.encode(text, convert_to_tensor=True).to(device)
        else:
            return model.encode(text, convert_to_tensor=True).to(device)
    
    # Define the ChromaDB path
    chroma_db_path = "./chroma_db"
    
    # Initialize a persistent ChromaDB client
    chroma_client = chromadb.PersistentClient(path=chroma_db_path)
    
    # Check if the collection already exists
    existing_collections = chroma_client.list_collections()
    if "temp_case_classification" in [col for col in existing_collections]:
        collection = chroma_client.get_collection("temp_case_classification")
    else:
        collection = chroma_client.create_collection(name="temp_case_classification")

        # Populate ChromaDB dynamically with classification categories and keywords
        for class_name, keywords in classification_dict.items():
            for keyword in keywords:
                embedding = encode_text(keyword)
                collection.add(embeddings=[embedding.cpu().tolist()], 
                               metadatas=[{"class_name": class_name}], 
                               ids=[keyword])
    
    # Encode the case text for retrieval (ensure it's on GPU)
    case_embedding = encode_text(case_text)

    # Retrieve the most relevant legal categories
    results = collection.query(query_embeddings=[case_embedding.cpu().tolist()], n_results=3)
    
    retrieved_classes = [
        res["class_name"] for res in results["metadatas"][0]
    ] if results["metadatas"] else []
    
    if not retrieved_classes:
        return "No relevant classification found"
    
    # Prepare relevant context for LLaMA 3
    retrieved_text = "\n".join(retrieved_classes)

    prompt = f"""
    You are an AI assistant trained in Supreme Court case classification. Given the following retrieved categories,
    classify the new case into the most appropriate issue area.

    Retrieved Relevant Categories:
    {retrieved_text}

    Please reply with the category only and nothing else.
    """

    # Query LLaMA 3 using the Ollama instance
    response = ollama.chat(model='llama3', messages=[{"role": "user", "content": prompt}])
    
    # Free GPU memory
    del case_embedding
    torch.cuda.empty_cache()

    return response['message']['content']


[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
# %%capture
# Process and classify cases
classified_cases = []

def process_cases():
    """
    Process all cases synchronously and classify them using the local API.
    """
    for idx, row in cases_df.iterrows():
        print(f"Processing case {idx + 1}/{len(cases_df)}...")

        # Classify the case
        class_name = classify_case_rag(row["text"], filtered_result)  # Pass the preprocessed text

        if class_name:
            classified_cases.append({"text": row["text"], "class_name": class_name})
        else:
            print(f"Failed to classify case {idx + 1}")
        # break

    
    print("Processing completed.")

# Run the case processing
process_cases()



Processing case 1/8419...
Using 2 GPUs for inference.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Processing case 2/8419...
Using 2 GPUs for inference.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Processing case 3/8419...
Using 2 GPUs for inference.


...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Processing case 5525/8419...
Using 2 GPUs for inference.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Processing case 5526/8419...


OutOfMemoryError: CUDA out of memory. Tried to allocate 46.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 12.12 MiB is free. Process 2242 has 14.73 GiB memory in use. Of the allocated memory 13.55 GiB is allocated by PyTorch, and 1.06 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# Save the results to a DataFrame
classified_df = pd.DataFrame(classified_cases)

# filling none for Nan values
classified_df.fillna({'class_name': 'None'}, inplace=True)

# Display the results
classified_df.head()

In [ ]:
classified_df.to_csv("/kaggle/working/classified_rag_llama3.csv")

In [ ]:
for rows in classified_df.iterrows():
    print(rows[1]['class_name'])
    break